# Save all data needed for plots to an output file

The output file will be readed in DASH or Superset and must contain all the columns used to produce the plots in the static version of the [blogpost](https://unicef.sharepoint.com/:w:/r/teams/ICTD-MagicBox/DocumentLibrary1/publications/blogposts/Blogpost%20-%20government%20measures.docx?d=wb33e4efb34fb44ae94d3f07a71879e95&csf=1&web=1&e=bfTegd)

In GeoPandas version 0.6.2, saving a geopandas DataFrame in a CSV file works (command `geodf.to_csv()`) but reading back the data from the CSV file does not work (command `geodf = gpd.read_file()`).
The file to save is a GeoJSON but can be changed at the bottom of the notebook.

In [11]:
import requests
import pandas as pd
import numpy as np
import geopandas as gpd
# import pickle
import os
# import shapefile
from datetime import datetime, timedelta
# import operator
# import math
# import copy
# from collections import Counter, defaultdict
# import matplotlib
# import matplotlib.pyplot as plt
# from matplotlib.collections import LineCollection
# from matplotlib.ticker import NullLocator, FuncFormatter
import matplotlib.colors as mcolors
import matplotlib.cm as mcm
import matplotlib.dates as mdates
# from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
# from mpl_toolkits.axes_grid1.colorbar import colorbar
# import matplotlib.gridspec as gridspec
# import seaborn as sns
# from descartes import PolygonPatch
# %matplotlib notebook

## change font
# matplotlib.rcParams['font.sans-serif'] = "Lato"
# matplotlib.rcParams['font.weight'] = "light"
# matplotlib.rcParams['font.size'] = 9
# matplotlib.rcParams['axes.labelweight'] = 'bold'

---

# <font color='red'> CHANGE `figures_dir` and `data_dir` </font>

In [12]:
cwd = os.getcwd()

# display(HTML("<style>.container { width:100% !important; }</style>"))

figures_dir = os.path.join(cwd, 'figures')
# data_dir = os.path.join(cwd, 'data')

data_dir = os.path.join(cwd, '../../data')

# Create local `data` and `figures` folder to store the data and figures respectively

# if not os.path.exists(data_dir):
#     os.makedirs(data_dir)

# if not os.path.exists(figures_dir):
#     os.makedirs(figures_dir)

## Download and save data from url

In [13]:
def download_data_from_url(url, folder_to_save, filename):
    
    req = requests.get(url)
    url_content = req.content

    with open(os.path.join(folder_to_save, filename), 'wb') as csv_file:
        csv_file.write(url_content)

### COVID-19 data from *Our World in Data*

Previously we used the dataset from ECDC but the daily reporting was discontinued (see below) so we switched to Our World in Data which sources them from John Hopkins Univesity

**WARNING**: From official [download page](https://www.ecdc.europa.eu/en/covid-19/data) of the dataset

> **ECDC switched to a weekly reporting schedule for the COVID-19 situation worldwide and in the EU/EEA and the UK on 17 December 2020. Hence, all daily updates have been discontinued from 14 December**. ECDC will publish updates on the number of cases and deaths reported worldwide and aggregated by week every Thursday. The weekly data will be available as downloadable files in the following formats: XLSX, CSV, JSON and XML.
>
> With the switch from daily to weekly reporting, ECDC will shift its Epidemic Intelligence (EI) resources from case counting to signal/event detection and resume its regular EI activities, which will include COVID-19 signal and event detection and analysis but also other potential threats.  


In [14]:
owid_covid_url = 'https://covid.ourworldindata.org/data/owid-covid-data.csv'

owid_filename = 'COVID-19_OWID.csv'

download_data_from_url(owid_covid_url, data_dir, owid_filename)

### Stringency Index by Oxford University

In [15]:
stringency_url = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_latest.csv'

stringency_filename = 'OxCGRT_latest.csv'

download_data_from_url(stringency_url, data_dir, stringency_filename)

### GDP per capita

Gross domestic product (GDP) per capita at purchasing power parity (PPP) (constant 2017 international dollars), from World Bank at https://data.worldbank.org/indicator/NY.GDP.PCAP.PP.KD

In [16]:
gdp_per_capita_url = 'https://api.worldbank.org/v2/en/indicator/NY.GDP.PCAP.PP.KD?downloadformat=excel'

gdp_per_capita_filename = 'gdp_per_capita.xls'

download_data_from_url(gdp_per_capita_url, data_dir, gdp_per_capita_filename)

---

# Load datasets

## COVID-19 data from *Our World in Data*

In [ ]:
covid_filename = 'COVID-19_OWID.csv'

df_cases = pd.read_csv(os.path.join(data_dir, covid_filename), parse_dates=['date'])

## Stringency Index for governmental policies 

In [ ]:
stringency_filename = 'OxCGRT_latest.csv'

df_ox = pd.read_csv(os.path.join(data_dir, stringency_filename), low_memory=False, parse_dates=['Date'])

## GDP per capita

In [ ]:
df_gdp_per_capita = pd.read_excel(os.path.join(data_dir, 'gdp_per_capita.xls'), sheet_name='Data', skiprows=3)

## Map of the world countries with boundaries

### GeoDataframe from geopandas dataset, contains a 'geometry' column of country borders polygons + other info

* Dataset `naturalearth_lowres` below is generated from a reference dataset downloaded from [here](http://www.naturalearthdata.com/downloads/110m-cultural-vectors/110m-admin-0-countries/)
* The script that generates the dataset `naturalearth_lowres` loaded in geopandas is [here](https://github.com/geopandas/geopandas/blob/master/geopandas/datasets/naturalearth_creation.py)

In [ ]:
gdf_world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# Remove Antarctica from GeoDataFrame
gdf_world = gdf_world[gdf_world['name']!="Antarctica"]

# Fix missing 'iso_a3' codes for countries with 'name'
display(gdf_world.loc[gdf_world['iso_a3'] == '-99'])

gdf_world.loc[gdf_world['name'] == 'France', 'iso_a3'] = 'FRA'
gdf_world.loc[gdf_world['name'] == 'Norway', 'iso_a3'] = 'NOR'
gdf_world.loc[gdf_world['name'] == 'Kosovo', 'iso_a3'] = 'XXK'
gdf_world.loc[gdf_world['name'] == 'N. Cyprus', 'iso_a3'] = 'CYP'
gdf_world.loc[gdf_world['name'] == 'Somaliland', 'iso_a3'] = 'SOM'
gdf_world.loc[gdf_world['name'] == 'W. Sahara', 'iso_a3'] = 'MAR'

Countries from geopandas dataframe not in World Bank GDP per capita dataset

In [ ]:
gdf_world.loc[~gdf_world['iso_a3'].isin(df_gdp_per_capita['Country Code'])]

Display entries from World Bank dataframe on GDP per capita which are not in geopandas gdf_world.

Mostly small islands or sovereign territories, even if there are exceptions of small states, the most notable being Andorra, Bahrain, Gibraltar, San Marino, Monaco, Liechtenstein, Singapore.

In [ ]:
# pd.set_option('display.max_rows', None)
# df_gdp_per_capita.loc[~df_gdp_per_capita['Country Code'].isin(gdf_world['iso_a3'])]

In [ ]:
pd.set_option('display.max_rows', 10)

### Official UN dataset of country boundaries

* GeoJSON downloaded from the UN Geospatial Hub [here](https://geoportal.dfs.un.org/arcgis/apps/sites/#/geohub/items/c4b9b7a3bfb644d19158220ae0decb65). 
(Can be reached from the [main page](https://geoservices.un.org/webapps/geohub/) of UN Geospatial Hub, click on 'Geodata', then on the link 'UN map carto geojson' or 'UN map carto shapefile', finally on 'Download' button top right).

* Alternatively, the 'UN Map' of the world provided by OCHA FISS can be downloaded from HDX after request at https://data.humdata.org/dataset/united-nations-map 

## Governmental measures on policy restrictions

Dates of physical distancing or lockdown measures implemented by governments were manually selected from ACAPS Excel file listing all measures in various areas (health, physical distancing, travels ban, lockdowns, economy, ...) from https://www.acaps.org/covid-19-government-measures-dataset

In [ ]:
def save_measures():

    measures = {'MYS': [(datetime(2020,3,18),'lockdown'),(datetime(2020,5,7),'measures\nloosened')],
                'IDN':[(datetime(2020,3,16),'physical\ndistancing\nenacted'), (datetime(2020,5,7),'resumption\ndomestic\nflights')
                       #(datetime(2020,4,27),'ban on going\nhome for\nRamadan')
                      ],
                'IND':[#(datetime(2020,3,5),'public\ngatherings\nlimited'),
                       #(datetime(2020,3,14),'emergency\nprotocols\ninvoked'),
                       (datetime(2020,3,24),'lockdown'), (datetime(2020,4,25),'shops\nreopening')],
                'MEX':[(datetime(2020,3,14),'public\nhealth\nmeasures'),
                       #(datetime(2020,3,30),'lockdown'),
                       (datetime(2020,5,18),'partial\nreopening')
                       #, (datetime(2020,6,1),'total\nreopening')
                      ],
                'NGA':[#(datetime(2020,3,23),'schools\nclosed\n(Lagos)'),
                       (datetime(2020,3,30),'domestic\ntravel\nrestrictions'),
                       (datetime(2020,6,2),'partial\nreopening')
                      ],
                'COL':[(datetime(2020,3,12),'health\nemergency\ndeclared'),
                       (datetime(2020,5,5),'partial\nreopening')],
                'CIV':[(datetime(2020,3,24),'national\ncurfew'),
                       (datetime(2020,5,8),'total\nreopening')],
                'MMR':[(datetime(2020,3,13),'suspension of\npublic gatherings'),(pd.NaT,'')],
                'MOZ':[(datetime(2020,3,23),'limit public\ngatherings'),
                       (datetime(2020,6,28),'international\nflights\nallowed')
                      ],
                'UKR':[(datetime(2020,3,15),'lockdown'),
                       (datetime(2020,5,22),'partial\nreopening')],
                'AUS':[(datetime(2020,3,29),'lockdown'),
                       (datetime(2020,5,22),'partial\nreopening')],
                'DEU':[(datetime(2020,3,16),'non-essential\npublic\nservices\nclosed'),
                       (datetime(2020,5,15),'internal\ntransport\nresumed')],
                'GBR':[(datetime(2020,3,24),'lockdown'),
                       (datetime(2020,5,10),'go to\nwork\nallowed')],
                'ITA':[(datetime(2020,3,23),'lockdown'),
                       (datetime(2020,5,10),'businesses\nreopening')],
                'SGP':[(datetime(2020,4,7),'limit public gatherings'),
                       (datetime(2020,6,19),'businesses reopening')]
               }
    
    
    abbr_to_name = {'CIV':"Côte d'Ivoire",
                'COL':'Colombia',
                'IDN':'Indonesia',
                'IND':'India',
                'MEX':'Mexico',
                'MMR':'Myanmar',
                'MOZ':'Mozambique',
                'MYS':'Malaysia',
                'NGA':'Nigeria',
                'UKR':'Ukraine',
                'AUS':'Australia',
                'DEU':'Germany',
                'GBR':'United Kingdom',
                'ITA':'Italy',
                'SGP':'Singapore',
                'ESP':'Spain',
                'SWE':'Sweden',
                'KOR':'South Korea',
                'IRL':'Ireland'
               }

    
    country_code = []
    start_date = []
    end_date = []
    start_string = []
    end_string = []
    country_name = []
    
    for cc,m in measures.items():
        country_code.append(cc)
        start_date.append(m[0][0])
        start_string.append(m[0][1])
        end_date.append(m[1][0])
        end_string.append(m[1][1])
        country_name.append(abbr_to_name[cc])
    
    df_measures = pd.DataFrame({'iso_a3':country_code,'country':country_name,
                                'start_date':start_date, 'start_string':start_string, 
                                'end_date':end_date, 'end_string':end_string},
                               columns = ['iso_a3', 'country', 'start_date', 'start_string', 'end_date', 'end_string'])
    
    df_measures.to_csv(os.path.join(data_dir, 'dates_government_measures.csv'), index=False)
    
    return 

In [ ]:
save_measures()

---

# Functions to save/write datasets for different plots

## Merge into a single dataframe the COVID-19 dataset and the Stringency Index dataset

### Preprocess COVID-19 dataset

* Keep only non-negative `cases` and `deaths`

In [ ]:
df_cases = df_cases.loc[df_cases['new_cases'] >= 0]
df_cases = df_cases.loc[df_cases['new_deaths'] >= 0]

* Calculate `cases` and `deaths` per 100k population

In [ ]:
df_cases['new_cases_per_100000'] = (df_cases['new_cases'].astype('float')*100000)/df_cases['population']
df_cases['new_deaths_per_100000'] = (df_cases['new_deaths'].astype('float')*100000)/df_cases['population']

* Calculate 7 day averages for each country

In [ ]:
def moving_average(s_input, window):
    return s_input.rolling(window=window, min_periods=1).mean()

In [ ]:
grouped_country_df_cases = df_cases.groupby('iso_code')
                                                            
sel_cols = ['new_cases', 'new_deaths', 'new_cases_per_100000', 'new_deaths_per_100000']

for col in sel_cols:
    df_cases[col+'_7_day_average'] = grouped_country_df_cases[col].apply(lambda x: moving_average(x,7))
    df_cases[col+'_7_day_average_daily_change'] = df_cases[col+'_7_day_average'].pct_change()*100

In [ ]:
# Check moving average applied to grouped_country_df_cases is what expected for a single contry

# df_cases_country = df_cases.loc[df_cases['iso_code'] == 'BRA']
# s_moving_avg = df_cases_country['new_cases'].rolling(window=7, min_periods=1).mean()
# display(s_moving_avg.equals(df_cases_country['new_cases_7_day_average']))

## Preprocess Stringency Index dataset

* The stringency index may have been produced at subnational level for few countries (Brazil, Canada, USA, UK), so subnational indexes must be excluded since would result in multiple values (national and subnationals) for the same country. We only want to keep the national stringency index.

In [ ]:
## Countries with subnational stringency index
# df_ox.loc[df_ox['RegionName'].notnull()][['CountryName']].drop_duplicates()

df_ox = df_ox.drop(df_ox.loc[df_ox['RegionName'].notnull()].index)

## Merging the Stringency Index into the COVID-19 dataframe

In [ ]:
columns_df_ox_keep = ['CountryCode','Date','StringencyIndexForDisplay']

columns_df_ox_merge = ['CountryCode', 'Date']
columns_df_cases_merge = ['iso_code','date']

df_merged_cases_stringency = pd.merge(df_cases, df_ox[columns_df_ox_keep], left_on=columns_df_cases_merge, right_on=columns_df_ox_merge, how='left')

Save later the dataset since we will insert a flag column to reproduce the heatmap in the blogpost.

---

## Number of countries vs first day of maximum stringency index

In [ ]:
def save_hist_countries_vs_date_max_stringency(df_input, freq_datespan, colormap):
    
    only_national = df_input['RegionName'].isnull()
    
    df_national = df_input.loc[only_national]
    grp_national = df_national.groupby('CountryName')
    
    colmax = grp_national['StringencyIndexForDisplay'].max() 
    
    list_first_date_max_stringency = []
    
    # df_date_edges = pd.date_range(start = df_input['Date'].min(), end = df_input['Date'].max(), freq = '7D')
    # list_date_edges = np.arange(df_input['Date'].min(), df_input['Date'].max(), dtype='datetime64[W]')

    min_date, max_date = df_input['Date'].min(), df_input['Date'].max()
    
    for name, group in grp_national:
        indx_max_stringency = group['StringencyIndexForDisplay'] == group['StringencyIndexForDisplay'].max()
        # Get the first day of max strinngency
        indx_first_date_max_stringency = group.loc[indx_max_stringency]['Date'].min()
        list_first_date_max_stringency.append(indx_first_date_max_stringency.to_datetime64())
    
    df_first_date_max_stringency = pd.Series(sorted(list_first_date_max_stringency), dtype='datetime64[ns]')
    
    # Check if the year is always the same, since df_first_date_max_stringency.dt.week is the week number of that year.
    # If there are different years, then a reference to this info should be included in the plot from the weekday
    if df_first_date_max_stringency.dt.year.nunique() == 1:

        bottom_date = '2020-03-01'
        top_date = '2020-08-30'
        
        ## pd.date_range(min_date, max_date, freq='W-MON') below starts from Sunday (excluded), could start from any other day (excluded) using freq='W-MON' for example. 
        ## If min_date it is after then the first week is incomplete and does not get inside the range
        # df_dates_range = pd.date_range(min_date, max_date, freq='W-MON')        
        res = df_first_date_max_stringency.groupby(pd.cut(df_first_date_max_stringency, 
                                                              pd.date_range(min_date, max_date, freq=freq_datespan))).count()

        aggr_date_range = pd.date_range(min_date, bottom_date, periods=1).append(pd.date_range(bottom_date, top_date, freq=freq_datespan)).append(pd.DatetimeIndex([max_date]))
        list_aggr_date_range = aggr_date_range.tolist()
        
        res_aggr = df_first_date_max_stringency.groupby(pd.cut(df_first_date_max_stringency, aggr_date_range)).count()

        # Splitting categorical index of res_aggr into columns
        df_to_save = res_aggr.index.categories.astype(str).to_series().str.split(', ', expand=True)
        df_to_save = df_to_save.rename(columns={0:'start_date',1:'end_date'}).replace("\]|\(", "", regex=True)
        
        # Adding counts for countries in interval dates from res_aggr
        df_to_save['num_countries'] = res_aggr.values
        
        ## Define discrete colors for the colorbar and for bars of the histogram plot.
        # Discard first and last dates since I don't want to plot them since they are at the limits of the range
        bounds = [mdates.date2num(i + timedelta(days=1)) for i in list_aggr_date_range[1:-1:2]]
        # mdates.date2num(datetime.strptime(i, "%Y-%m-%d"))
        out_bounds_min, out_bounds_max = mdates.date2num(list_aggr_date_range[0]), mdates.date2num(list_aggr_date_range[-1])
        
        cmap = mcm.get_cmap(colormap)  # define the colormap
        
        # extract all colors from the .jet map
        cmaplist = [cmap(i) for i in range(cmap.N)]

        # create the new map
        custom_cmap = mcolors.LinearSegmentedColormap.from_list('Custom cmap', cmaplist, cmap.N)

        # define the bins and normalize
        norm = mcolors.BoundaryNorm(bounds, cmap.N)
            
        # Define colors for the bars from the custom colormap. Bars have the same color 2 by 2 since once bar is one week while the colormap has one color for 2 weeks.
        custom_colors = [custom_cmap(norm(mdates.date2num(c.left + timedelta(days=1)))) for c in res_aggr.index.categories]            
        # custom_colors[0] = mcolors.to_rgba('0.9')
        custom_colors[0] = mcolors.to_rgba('0.7')
        custom_colors[-1] = mcolors.to_rgba('0.7')
        
        df_to_save['color'] = [mcolors.to_hex(c) for c in custom_colors]
        
        save_path = os.path.join(data_dir, 'histo_num_countries_vs_first_day_max_stringency.csv')

        df_to_save.to_csv(save_path, index=False)

In [ ]:
save_hist_countries_vs_date_max_stringency(df_ox.loc[df_ox['Date'] < '2020-10-16'], 'W', 'inferno_r') #'viridis', 'cividis', 'plasma'

---

## Geographical maps of countries colored according to a given value/indicator

In [ ]:
def save_geodataframe_first_day_max_stringency_cases_per_100k(gdf_input, df_stringency, df_cases, col_df_cases, freq_datespan, colormap_first_day_max_stringency):    
    
    only_national = df_stringency['RegionName'].isnull()
    
    df_stringency_national = df_stringency.loc[only_national]
    grp_stringency_national = df_stringency_national.groupby('CountryCode')
        
    # df_date_edges = pd.date_range(start = df_input['Date'].min(), end = df_input['Date'].max(), freq = '7D')
    # list_date_edges = np.arange(df_input['Date'].min(), df_input['Date'].max(), dtype='datetime64[W]')
    
    min_date, max_date = df_stringency['Date'].min(), df_stringency['Date'].max()
    
    # Create a DataFrame with country as index and first day of max stringency as values in the column
    
    list_first_day_max_stringency = []
    list_country_code = []
    list_total_cases_per_100k_at_first_day_max_stringency = []
    list_max_stringency = []
    list_first_day_max_moving_avg_cases = []
    # Catch the date of stringency values close to the maximum (within max stringency - 10)
    list_first_day_sim_max_stringency = []

    
    for name, group in grp_stringency_national:
        indx_max_stringency = group['StringencyIndexForDisplay'] == group['StringencyIndexForDisplay'].max()
        list_max_stringency.append(group['StringencyIndexForDisplay'].max())
        # Get the first day of max strinngency
        list_country_code.append(name)
        indx_first_day_max_stringency = group.loc[indx_max_stringency]['Date'].min()
        list_first_day_max_stringency.append(indx_first_day_max_stringency.to_datetime64())
        
        df_cases_name = df_cases.loc[df_cases['iso_code'] == name]
        indx_dates_before_date_max_stringency = df_cases_name['date'] <= indx_first_day_max_stringency
        total_cases_per_100k = df_cases_name.loc[indx_dates_before_date_max_stringency][col_df_cases].sum()
        list_total_cases_per_100k_at_first_day_max_stringency.append(total_cases_per_100k)

    df_first_day_max_stringency = pd.DataFrame({'first_day_max_stringency':list_first_day_max_stringency,
                                                 'first_day_max_stringency_numerical': [mdates.date2num(i) for i in list_first_day_max_stringency],
                                                 'total_cases_per_100k_at_first_day_max_stringency' : list_total_cases_per_100k_at_first_day_max_stringency,
                                                 'max_stringency' : list_max_stringency
                                                },
                                                index=list_country_code)

    ## Easy way to put one more column in the GeoDataFrame then used to plot borders and colors/value from the new columns
    ## Merging df_first_day_max_stringency on gdf_input
    gdf_merged = gdf_input.merge(df_first_day_max_stringency, left_on = 'iso_a3', right_on = df_first_day_max_stringency.index, how='left')
    
    # Check if the year is always the same, since df_first_day_max_stringency.dt.week is the week number of that year.
    # If there are different years, then a reference to this info should be included in the plot from the weekday
    if gdf_merged['first_day_max_stringency'].dt.year.nunique() == 1:

        bottom_date = '2020-03-01'
        top_date = '2020-08-30'
        
        ## pd.date_range(min_date, max_date, freq='W-MON') below starts from Sunday (excluded), could start from any other day (excluded) using freq='W-MON' for example. 
        ## If min_date it is after then the first week is incomplete and does not get inside the range
        # df_dates_range = pd.date_range(min_date, max_date, freq='W-MON')        
        # res = gdf_merged['first_day_max_stringency'].groupby(pd.cut(gdf_merged['first_day_max_stringency'], 
        #                                                       pd.date_range(min_date, max_date, freq=freq_datespan))).count()

        
        ## All dates including single group per all dates before bottom_date and another after top_date
        aggr_date_range = pd.date_range(min_date, bottom_date, periods=1).append(pd.date_range(bottom_date, top_date, freq=freq_datespan)).append(pd.DatetimeIndex([max_date]))
        list_aggr_date_range = aggr_date_range.tolist()
        res_aggr = gdf_merged['first_day_max_stringency'].groupby(pd.cut(gdf_merged['first_day_max_stringency'], aggr_date_range)).count()
        ## Define discrete colors for the colorbar and for bars of the histogram plot.
        ## Discard first and last dates since I don't want to plot them since they are at the limits of the range
        bounds_first_day_max_stringency = [mdates.date2num(i + timedelta(days=1)) for i in list_aggr_date_range[1:-1:2]]
        out_bounds_first_day_max_stringency_min, out_bounds_first_day_max_stringency_max = mdates.date2num(list_aggr_date_range[0]), mdates.date2num(list_aggr_date_range[-1])            
        
        # Instead of the interval in days corresponding to the week, get only the corresponding Monday to then substitute the x ticks
        # monday_of_week = [(c.left + timedelta(days=1)) for c in res.index.categories]
        # monday_of_week_str = [date.strftime('%d.%m') for date in monday_of_week]

        ## Group dates of max stringency by week number of the year
        # count_first_day_max_stringency_per_week = df_first_day_max_stringency.groupby(df_first_day_max_stringency.dt.week).count()
        # count_first_day_max_stringency_per_week.plot(kind='bar', ax=ax)
        # display(df_first_day_max_stringency.apply(lambda x: x - timedelta(days=x.weekday())))

        cmap_first_day_max_stringency = mcm.get_cmap(colormap_first_day_max_stringency)  # define the colormap
        # extract all colors from the .jet map
        cmaplist_first_day_max_stringency = [cmap_first_day_max_stringency(i) for i in range(cmap_first_day_max_stringency.N)]

        # create the new map
        custom_cmap_first_day_max_stringency = mcolors.LinearSegmentedColormap.from_list('Custom cmap', cmaplist_first_day_max_stringency, cmap_first_day_max_stringency.N)

        # define the bins and normalize
        norm_first_day_max_stringency = mcolors.BoundaryNorm(bounds_first_day_max_stringency, cmap_first_day_max_stringency.N)

        ## Define colors for the bars from the custom colormap. Bars have the same color 2 by 2 since once bar is one week while the colormap has one color for 2 weeks.
        custom_colors_first_day_max_stringency = [custom_cmap_first_day_max_stringency(norm_first_day_max_stringency(mdates.date2num(c.left + timedelta(days=1)))) for c in res_aggr.index.categories]            
        
        custom_colors_first_day_max_stringency[0] = mcolors.to_rgba('0.7')
        custom_colors_first_day_max_stringency[-1] = mcolors.to_rgba('0.7')
        
        gdf_merged['color_first_day_max_stringency'] = gdf_merged['first_day_max_stringency_numerical'].apply(lambda x: mcolors.to_hex(custom_cmap_first_day_max_stringency(norm_first_day_max_stringency(x))))
        
        ## Plot total cases per 100k population at first day of max stringency
        indx_nnz = gdf_merged['total_cases_per_100k_at_first_day_max_stringency'] > 0
        min_total_cases_per_100k = gdf_merged.loc[indx_nnz]['total_cases_per_100k_at_first_day_max_stringency'].min()
        max_total_cases_per_100k = gdf_merged.loc[indx_nnz]['total_cases_per_100k_at_first_day_max_stringency'].max()
        
        indx_null = gdf_merged['first_day_max_stringency_numerical'].isnull()
        indx_ls_min_bounds = gdf_merged['first_day_max_stringency_numerical'] < min(bounds_first_day_max_stringency)
        indx_gt_max_bounds = gdf_merged['first_day_max_stringency_numerical'] > max(bounds_first_day_max_stringency)

        ## Substituting dates outside of the range with null values
        gdf_merged.loc[indx_ls_min_bounds,'first_day_max_stringency_numerical'] = np.nan
        gdf_merged.loc[indx_gt_max_bounds,'first_day_max_stringency_numerical'] = np.nan
        
        gdf_merged.loc[indx_ls_min_bounds,'first_day_max_stringency'] = pd.NaT
        gdf_merged.loc[indx_gt_max_bounds,'first_day_max_stringency'] = pd.NaT

        gdf_merged.loc[indx_ls_min_bounds,'total_cases_per_100k_at_first_day_max_stringency'] = np.nan
        gdf_merged.loc[indx_gt_max_bounds,'total_cases_per_100k_at_first_day_max_stringency'] = np.nan
    
        # save_path = os.path.join(data_dir, "geodata_cases_max_stringency.geojson")
        # gdf_merged.to_file(save_path, driver='GeoJSON')
        
    return gdf_merged

In [ ]:
gdf_first_day_max_stringency_cases_per_100k = save_geodataframe_first_day_max_stringency_cases_per_100k(gdf_world, df_ox.loc[df_ox['Date'] < '2020-10-16'], df_cases, 'new_cases_per_100000', 'W', 'inferno_r')

---

In [ ]:
def save_geodataframe_delay_max_cases_per_100k_and_first_day_max_stringency(gdf_input, df_stringency, df_cases, col_df_cases, bottom_date, top_date, bin_width_days, colormap):
        
    only_national = df_stringency['RegionName'].isnull()
    
    df_stringency_national = df_stringency.loc[only_national]
    df_stringency_national = df_stringency_national.loc[(df_stringency_national['Date'] >= bottom_date) & (df_stringency_national['Date'] <= top_date)]
    grp_stringency_national = df_stringency_national.groupby('CountryCode')
    
    df_cases = df_cases.loc[(df_cases['date'] >= bottom_date) & (df_cases['date'] <= top_date)] 
    
    # df_date_edges = pd.date_range(start = df_input['Date'].min(), end = df_input['Date'].max(), freq = '7D')
    # list_date_edges = np.arange(df_input['Date'].min(), df_input['Date'].max(), dtype='datetime64[W]')
    
    # min_date, max_date = df_stringency['Date'].min(), df_stringency['Date'].max()
    
    # Create a DataFrame with country as index and first day of max stringency as values in the column
    
    list_first_day_max_stringency = []
    list_country_code = []
    list_first_day_max_moving_avg_cases = []
    list_max_stringency = []
    # Catch the date of stringency values close to the maximum (within max stringency - 10)
    list_first_day_sim_max_stringency = []
    
    for name, group in grp_stringency_national:
        max_stringency = group['StringencyIndexForDisplay'].max()
        indx_max_stringency = group['StringencyIndexForDisplay'] == max_stringency
        list_max_stringency.append(max_stringency)
        list_country_code.append(name)
        ## Get the first day of max strinngency
        indx_first_day_max_stringency = group.loc[indx_max_stringency]['Date'].min()
        list_first_day_max_stringency.append(indx_first_day_max_stringency.to_datetime64())
        ## Get first day with stringency similar to the maximum one
        ## Before filtering on max_stringency >= 50 but Japan (which did a good job) is < 50, so I removed the if condition
        # if max_stringency >= 50:
        indx_sim_max_stringency = group['StringencyIndexForDisplay'] > max_stringency-15
        indx_first_day_sim_max_stringency = group.loc[indx_sim_max_stringency]['Date'].min()
        list_first_day_sim_max_stringency.append(indx_first_day_sim_max_stringency.to_datetime64())

        #
        df_cases_name = df_cases.loc[df_cases['iso_code'] == name]
        if len(df_cases_name) == 0:
            day_max_moving_avg_cases = pd.NaT
        else:
            df_cases_name_moving_avg = df_cases_name[col_df_cases].rolling(window=7, min_periods=1).mean()
            indx_day_max_moving_avg_cases = df_cases_name_moving_avg.idxmax()
            day_max_moving_avg_cases = df_cases_name.loc[indx_day_max_moving_avg_cases]['date']
        
        list_first_day_max_moving_avg_cases.append(day_max_moving_avg_cases.to_datetime64())

    df_first_day_max_stringency = pd.DataFrame({'first_day_max_stringency':list_first_day_max_stringency,
                                                 'first_day_max_stringency_numerical': [mdates.date2num(i) for i in list_first_day_max_stringency],
                                                 'first_day_max_' + col_df_cases : list_first_day_max_moving_avg_cases,
                                                 'max_stringency' : list_max_stringency,
                                                 'first_day_sim_max_stringency':list_first_day_sim_max_stringency
                                                },
                                                index=list_country_code)
    
    ## Adding column for the difference in days between max cases and max stringency
    df_first_day_max_stringency['delay_day_max_' + col_df_cases + '_first_day_max_stringency'] = df_first_day_max_stringency['first_day_max_' + col_df_cases] - df_first_day_max_stringency['first_day_max_stringency']
    df_first_day_max_stringency['delay_day_max_' + col_df_cases + '_first_day_max_stringency_numerical'] = df_first_day_max_stringency['delay_day_max_' + col_df_cases + '_first_day_max_stringency'].dt.days
    # display(df_first_day_max_stringency.loc[df_first_day_max_stringency['delay_day_max_' + col_df_cases + '_first_day_max_stringency_numerical'] < 0])
    
    df_first_day_max_stringency['delay_day_max_' + col_df_cases + '_first_day_sim_max_stringency'] = df_first_day_max_stringency['first_day_max_' + col_df_cases] - df_first_day_max_stringency['first_day_sim_max_stringency']
    df_first_day_max_stringency['delay_day_max_' + col_df_cases + '_first_day_sim_max_stringency_numerical'] = df_first_day_max_stringency['delay_day_max_' + col_df_cases + '_first_day_sim_max_stringency'].dt.days

    ## DELETE the reduntant columns 
    # 'delay_day_max_' + col_df_cases + '_first_day_max_stringency', 'delay_day_max_' + col_df_cases + '_first_day_sim_max_stringency'
    ## since they have been converted in the numerical counterpart of number of days and are not used later,
    ## plus they contain the type pandas._libs.tslibs.timedeltas.Timedelta which cannot be converted/saved into a GeoJSON
    
    columns_to_drop = ['delay_day_max_' + col_df_cases + '_first_day_max_stringency', 'delay_day_max_' + col_df_cases + '_first_day_sim_max_stringency']
    df_first_day_max_stringency = df_first_day_max_stringency.drop(columns=columns_to_drop)
    
    # display(df_first_day_max_stringency.loc[df_first_day_max_stringency['delay_day_max_' + col_df_cases + '_first_day_sim_max_stringency_numerical'] < 0])
    
    col_to_plot = 'delay_day_max_' + col_df_cases + '_first_day_sim_max_stringency_numerical'
    col_ref = 'first_day_sim_max_stringency'
    
    ## Other valid pair is
    # col_to_plot = 'delay_day_max_' + col_df_cases + '_first_day_max_stringency_numerical'
    # col_ref = 'first_day_max_stringency'
    
    ## Plot histogram of delay in days
    # fig_hist, ax_hist = plt.subplots(1,1,figsize=(6,4))
    # df_first_day_max_stringency[col_to_plot].hist(bins=np.arange(df_first_day_max_stringency[col_to_plot].min(), df_first_day_max_stringency[col_to_plot].max()+1, 7), ax=ax_hist)
    
    ## Easy way to put one more column in the GeoDataFrame then used to plot borders and colors/value from the new columns
    ## Merging df_first_day_max_stringency on gdf_input
    gdf_merged = gdf_input.merge(df_first_day_max_stringency, left_on = 'iso_a3', right_on = df_first_day_max_stringency.index, how='left')
    
    # Check if the year is always the same, since df_first_day_max_stringency.dt.week is the week number of that year.
    # If there are different years, then a reference to this info should be included in the plot from the weekday
    if gdf_merged[col_ref].dt.year.nunique() == 1:
        
        ## pd.date_range(min_date, max_date, freq='W-MON') below starts from Sunday (excluded), could start from any other day (excluded) using freq='W-MON' for example. 
        ## If min_date it is after then the first week is incomplete and does not get inside the range
        # df_dates_range = pd.date_range(min_date, max_date, freq='W-MON')        
        # res = gdf_merged['first_day_max_stringency'].groupby(pd.cut(gdf_merged['first_day_max_stringency'], 
        #                                                       pd.date_range(min_date, max_date, freq=freq_datespan))).count()

        
        ## All dates including single group per all dates before bottom_date and another after top_date
        # aggr_date_range = pd.date_range(min_date, bottom_date, periods=1).append(pd.date_range(bottom_date, top_date, freq=freq_datespan)).append(pd.DatetimeIndex([max_date]))
        # list_aggr_date_range = aggr_date_range.tolist()
        # res_aggr = gdf_merged['first_day_max_stringency'].groupby(pd.cut(gdf_merged['first_day_max_stringency'], aggr_date_range)).count()        
        ## Define discrete colors for the colorbar and for bars of the histogram plot.
        # Discard first and last dates since I don't want to plot them since they are at the limits of the range
        # bounds = [mdates.date2num(i + timedelta(days=1)) for i in list_aggr_date_range[1:-1:2]]
        # mdates.date2num(datetime.strptime(i, "%Y-%m-%d"))
        # out_bounds_min, out_bounds_max = mdates.date2num(list_aggr_date_range[0]), mdates.date2num(list_aggr_date_range[-1])            

        
        cmap = mcm.get_cmap(colormap)  # define the colormap
        # extract all colors from the .jet map
        cmaplist = [cmap(i) for i in range(cmap.N)]

        # create the new map
        custom_cmap = mcolors.LinearSegmentedColormap.from_list('Custom cmap', cmaplist, cmap.N)
        
        ## Manually modified the upper limit since only 1 country has 120 days delay, all others are below 105
        # bounds = np.arange(0, df_first_day_max_stringency[col_to_plot].max(), bin_width_days)
        bounds = np.arange(0, 106, bin_width_days)
        
        # define the bins and normalize
        norm = mcolors.BoundaryNorm(bounds, cmap.N)
        
        # Instead of the interval in days corresponding to the week, get only the corresponding Monday to then substitute the x ticks
        # monday_of_week = [(c.left + timedelta(days=1)) for c in res.index.categories]
        # monday_of_week_str = [date.strftime('%d.%m') for date in monday_of_week]
        
        ## Plotting
        
        # fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(12,5))

        ## Group dates of max stringency by week number of the year
        # count_first_day_max_stringency_per_week = df_first_day_max_stringency.groupby(df_first_day_max_stringency.dt.week).count()
        # count_first_day_max_stringency_per_week.plot(kind='bar', ax=ax)
        # display(df_first_day_max_stringency.apply(lambda x: x - timedelta(days=x.weekday())))

        ## Define colors for the bars from the custom colormap. Bars have the same color 2 by 2 since once bar is one week while the colormap has one color for 2 weeks.
        # custom_colors = [custom_cmap(norm(mdates.date2num(c.left + timedelta(days=1)))) for c in res_aggr.index.categories]            
        
        # indx_nnz = (gdf_merged['first_day_max_' + col_df_cases] - gdf_merged['first_day_max_stringency']) > 0
        
        indx_null = gdf_merged[col_ref].isnull()
        # indx_ls_min_bounds = gdf_merged['first_day_max_stringency_numerical'] < mdates.date2num(datetime.strptime(bottom_date, "%Y-%m-%d"))
        # indx_gt_max_bounds = gdf_merged['first_day_max_stringency_numerical'] > mdates.date2num(datetime.strptime(top_date, "%Y-%m-%d"))
        indx_ls_min_bounds = gdf_merged[col_ref] < bottom_date
        indx_gt_max_bounds = gdf_merged[col_ref] > top_date


        ## Plot countries colored in grey if out of the dates in bounds
        # gdf_merged.loc[indx_ls_min_bounds].plot(ax=ax, color='0.9', edgecolor='grey', linewidth=0.5)
        
        ## Do not plot, instead replace null values
        # if indx_ls_min_bounds.sum() > 0:
        #     gdf_merged.loc[indx_ls_min_bounds].plot(ax=ax, color='red', edgecolor='grey', linewidth=0.5)
        # if indx_gt_max_bounds.sum() > 0:
        #    gdf_merged.loc[indx_gt_max_bounds].plot(ax=ax, color='black', edgecolor='grey', linewidth=0.5)
        
        gdf_merged.loc[indx_ls_min_bounds][col_to_plot] = np.nan
        gdf_merged.loc[indx_gt_max_bounds][col_to_plot] = np.nan

        gdf_merged.loc[indx_ls_min_bounds][col_to_plot.strip("_numerical")] = pd.NaT
        gdf_merged.loc[indx_gt_max_bounds][col_to_plot.strip("_numerical")] = pd.NaT
        
        # countries_not_to_plot_cases = gdf_merged.loc[gdf_merged['first_day_max_stringency_numerical'].isnull()]['iso_a3'].values().tolist() + gdf_merged.loc[gdf_merged['first_day_max_stringency_numerical'] < min(bounds)]['iso_a3'].values().tolist() + gdf_merged.loc[gdf_merged['first_day_max_stringency_numerical'] > max(bounds)]['iso_a3'].values().tolist()
        
        ## Plot colors by delay between max cases per 100k population and first day of max stringency
        # gdf_merged.loc[~(indx_ls_min_bounds | indx_gt_max_bounds)].plot(column=col_to_plot,
        #                                                                 ax=ax,
        #                                                                 cmap=custom_cmap,
        #                                                                 norm=norm,
        #                                                                 edgecolor='grey', linewidth=0.5)
        
        
        
        ## Substituting the line below for coloring with drawing hatches
        # gdf_merged.loc[indx_null].plot(ax=ax, color='0.7', edgecolor='grey', linewidth=0.5)
        # matplotlib.rcParams['hatch.linewidth'] = 0.25
        # gdf_merged.loc[indx_null].apply(lambda x : ax.add_patch(PolygonPatch(x['geometry'],
        #                                                                      fill=False,
        #                                                                      edgecolor='grey',
        #                                                                      hatch='|'*10+'-'*10,
        #                                                                      linewidth=0.5)), axis=1)
        
        # divider = make_axes_locatable(ax)
        # cax = divider.append_axes("right", size="2%", pad=0)

        # colbar = matplotlib.colorbar.ColorbarBase(cax, 
        #                                           cmap=custom_cmap, 
        #                                           norm=norm, 
        #                                           spacing='uniform', 
        #                                           ticks=bounds)

        # custom_cmap.set_over('cyan')
        custom_cmap.set_under('mediumseagreen')
        
        # matplotlib.rcParams['hatch.linewidth'] = 0.25
        # gdf_merged.loc[gdf_merged[col_to_plot]<0].apply(lambda x : ax.add_patch(PolygonPatch(x['geometry'],
        #                                                                                      fill=False,
        #                                                                                      edgecolor='grey',
        #                                                                                      hatch='|'*6+'-'*6,
        #                                                                                      linewidth=0.5)), axis=1)

        
        gdf_merged['color_delay_sim_max_stringency'] = gdf_merged[col_to_plot].apply(lambda x: mcolors.to_hex(custom_cmap(norm(x))))
        
        # colbar.ax.tick_params(labelsize=ticksize)
        # colbar.ax.set_title('Num. of days', fontsize=ticksize, weight='bold')

  
        # ax.set_title('Delay between day of maximum cases and first day of high stringency index', weight='bold', fontsize=fs)
    
        ## Hide ticks and labels
        # ax.axis('off')

        # fig_path = os.path.join(figures_dir, 'mobility_cases_stringency', fig_name)
        # fig.tight_layout()
        # fig.savefig(fig_path, dpi=300, bbox_inches=0, pad=0)
    
        # save_path = os.path.join(data_dir, "geodata_delay_days_max_cases_high_stringency.geojson")
        # gdf_merged.to_file(save_path, driver='GeoJSON')        
    
    return gdf_merged, bottom_date, top_date

In [ ]:
gdf_delay, bottom_date, top_date = save_geodataframe_delay_max_cases_per_100k_and_first_day_max_stringency(gdf_world, 
                                                                        df_ox, 
                                                                        df_cases, 'new_cases_per_100000', 
                                                                        '2020-03-01', '2020-06-30', 
                                                                        7, 
                                                                        'plasma_r')

**Select columns to rename in order to later merge them in a single geodataframe**

In [ ]:
col_df_cases = 'new_cases_per_100000'

cols_rename_gdf_delay = ['first_day_max_stringency', 'first_day_max_stringency_numerical', 
                         'first_day_max_' + col_df_cases, 
                         'max_stringency', 
                         'first_day_sim_max_stringency', 
                         # 'delay_day_max_' + col_df_cases + '_first_day_max_stringency',
                         'delay_day_max_' + col_df_cases + '_first_day_max_stringency_numerical',
                         # 'delay_day_max_' + col_df_cases + '_first_day_sim_max_stringency',
                         'delay_day_max_' + col_df_cases + '_first_day_sim_max_stringency_numerical',
                         'color_delay_sim_max_stringency']

dict_cols_rename_gdf_delay = {k:k + "_{0}_{1}".format(bottom_date, top_date) for k in cols_rename_gdf_delay}

gdf_delay = gdf_delay.rename(columns = dict_cols_rename_gdf_delay)

### Merge geodataframes created in the 2 functions `save_geodataframe_` above 

In [ ]:
gdf_all_info = pd.merge(gdf_first_day_max_stringency_cases_per_100k, 
                        gdf_delay[list(dict_cols_rename_gdf_delay.values()) + ['iso_a3']], 
                        left_on = 'iso_a3', 
                        right_on = 'iso_a3', how='left')

save_path = os.path.join(data_dir, "geodata_COVID-19_stringency_delay.geojson")
gdf_all_info.to_file(save_path, driver='GeoJSON')

---

## Heatmap of cases_per_100000 and stringency for high, middle and low income countries (10 countries per each group)

In [ ]:
def add_gdp_per_capita_categories(df_input, df_gdp_per_capita, population_threshold):

    # date_min = datetime(2020, 1, 1)
    # date_max = datetime(datetime.now().year,datetime.now().month,datetime.now().day) - timedelta(days=1)
    # date_max = datetime(2020, 10, 16) - timedelta(days=1)
    
    # dd_date_min, mm_date_min = date_min.split('-')[2], date_min.split('-')[1]
    # dd_mm_date_min = "-".join([dd_date_min, mm_date_min])

    # dd_date_max, mm_date_max = date_max.split('-')[2], date_max.split('-')[1]
    # dd_mm_date_max = "-".join([dd_date_max, mm_date_max])

    
    # Not all the countries (e.g. territories) from gdf_input are present in df_stringency or df_cases, so keep only those that are there!
    # indx_countries_in_stringeny_cases = gdf_input['iso_a3'].isin(df_stringency['CountryCode']) & gdf_input['iso_a3'].isin(df_cases['iso_code'])
    # gdf_plot = gdf_input.loc[indx_countries_in_stringeny_cases]

    indx_countries_stringency_cases_notnull = df_input['iso_code'].notnull() & df_input['CountryCode'].notnull()
    df_countries_stringency_cases_notnull = df_input.loc[indx_countries_stringency_cases_notnull]

    df_unique_countries = df_countries_stringency_cases_notnull.drop_duplicates(subset=['iso_code', 'population'])
    
    ## Filter on country population
    # gdf_plot = gdf_plot.loc[gdf_plot['pop_est'] > 1*(10**6)]
    df_subset = df_unique_countries.loc[df_unique_countries['population'] > population_threshold]

    # gdf_plot['gdp_per_capita'] = gdf_plot['gdp_md_est'] / gdf_plot['pop_est']
    ## Add GDP per capita from World Bank in 2019
    # gdf_plot['gdp_per_capita'] = gdf_plot['iso_a3'].map(pd.Series(df_gdp_per_capita['2019'].values, index=df_gdp_per_capita['Country Code']))
    df_subset['gdp_per_capita'] = df_subset['iso_code'].map(pd.Series(df_gdp_per_capita['2019'].values, index=df_gdp_per_capita['Country Code']))
    
    ## Compute division in high, low, and middle income countries
    df_subset['category_gdp_per_capita'] = pd.qcut(df_subset['gdp_per_capita'], 3, labels=['poor', 'middle', 'rich']) # retbins=True,
        
    ## Store in df_input 'category_gdp_per_capita' per each country
    df_input['category_gdp_per_capita'] = df_input['iso_code'].map(pd.Series(df_subset['category_gdp_per_capita'].values, index=df_subset['iso_code']))
        
    ## Add a column which is True if the country has been selected to be plotted in the heatmap, False otherwise
    df_input['country_to_plot'] = False
    
    for i, gdp_cat in enumerate(['rich', 'middle', 'poor']):
        
        df_subset_gdp_cat = df_subset.loc[df_subset['category_gdp_per_capita'] == gdp_cat]
        countries_gdp_cat = df_subset_gdp_cat['iso_code']
        df_selected_countries = df_subset_gdp_cat.nlargest(10, 'gdp_per_capita', keep='all')
        top_gdp_per_capita_countries_code = df_selected_countries['iso_code']
        
        if gdp_cat == 'rich':
            # Manually adding a bunch of countries for high-income
            list_index = []
            list_sel_ccs = ['ITA', 'DEU', 'ESP', 'GBR', 'FRA']
            for sel_cc in list_sel_ccs:
                list_index.append(countries_gdp_cat.loc[countries_gdp_cat == sel_cc].index[0])
            top_gdp_per_capita_countries_code = top_gdp_per_capita_countries_code.drop(top_gdp_per_capita_countries_code.loc[top_gdp_per_capita_countries_code == "QAT"].index, axis=0)
            top_gdp_per_capita_countries_code = top_gdp_per_capita_countries_code.append(pd.Series(list_sel_ccs, index=list_index), verify_integrity=True)
                
        df_input.loc[df_input['iso_code'].isin(top_gdp_per_capita_countries_code), 'country_to_plot'] = True
        
        # print(i, df_selected_countries['countriesAndTerritories'].values.tolist())  
        
    return df_input

In [ ]:
df_merged_cases_stringency = add_gdp_per_capita_categories(df_merged_cases_stringency, df_gdp_per_capita, 1*(10**6))

---

# Save merged datasets for COVID-19 and stringency index

In [ ]:
dict_column_rename = {'StringencyIndexForDisplay':'stringency_index'}

df_merged_cases_stringency = df_merged_cases_stringency.rename(columns = dict_column_rename)

df_merged_cases_stringency.to_csv(os.path.join(data_dir, 'COVID-19_stats_stringency_index.csv'), index=False)
                                  # columns=columns_to_save_df_merged_cases_stringency,